## Introduction - The case for Lisbon

Regarding Lisbon I will try to map the parish of the city against foursquare in order to see what are the main activities of each parish. The purpose is to help to decide in which parish should I open a restaurant or any other kind of business that I may considerer.

##### For this analysis I will study the viability of opening a seafood restaurant. The target of my restaurant will be mostly tourists.

The first obstacule that I may find is to get the GPS coordenates of all the parishes of the city.

A limitation of this analysis that I may encounter is that all the conclusions will be taken without financial values (namely price of acquistion and/ or rental of the space).

This exercise could be helpfull to anyone interested in open a new business. 

## The Data - Lisbon Neighborhoods

For this assignement the data was collected from a website:

https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/Limites_Cartografia/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=true&outFields=*&returnGeometry=false&returnCentroid=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=html&token=

In [1]:
# libraries to import
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [26]:
# scrapping and cleaning the data
PageToScrape = requests.get('https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/Limites_Cartografia/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=true&outFields=*&returnGeometry=false&returnCentroid=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=html&token=').text
PageToScrape = BeautifulSoup(PageToScrape, 'xml')
PageToScrape = PageToScrape.find_all('table',{'class':'ftrTable'})
table = []
for row in PageToScrape:
    table.append([t.text.strip() for t in row.find_all('td')])
#df = pd.DataFrame(table, columns=['OBJECTID', 'NOME'])
df = pd.DataFrame(table)
df = df.drop([0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], axis=1)
df.head()

,5,21
0,Santo António,Point:\nX: -1018440.6546671605\nY: 4681834.853...
1,Parque das Nações,Point:\nX: -1012503.9786498934\nY: 4689466.068...
2,Marvila,Point:\nX: -1014226.7638454303\nY: 4685629.513...
3,Ajuda,Point:\nX: -1023989.21146323\nY: 4680527.21680...
4,Areeiro,Point:\nX: -1016739.3330696875\nY: 4684693.942...


In [27]:
# scrapping and cleaning the data
df[21] = df[21].str.replace("Point:\n", "")
# new data frame with split value columns 
new = df[21].str.split("\n", n = 1, expand = True) 
# making separate first name column from new data frame 
df["X"]= new[0] 
# making separate last name column from new data frame 
df["Y"]= new[1]
# Dropping old Name columns 
df.drop(columns =[21], inplace = True) 
df["X"] = df["X"].str.replace("X: ", "")
df["Y"] = df["Y"].str.replace("Y: ", "")
df.rename(columns={5: 'Freguesia'}, inplace=True)
df.head()

,Freguesia,X,Y
0,Santo António,-1018440.6546671605,4681834.85372911
1,Parque das Nações,-1012503.9786498934,4689466.0681953141
2,Marvila,-1014226.7638454303,4685629.5130308038
3,Ajuda,-1023989.21146323,4680527.2168011209
4,Areeiro,-1016739.3330696875,4684693.9425387029


Form this page I was able to get the parishes of Lisbon and there centroid, not scrapped. The next step was to convert to X and Y associated with each parish.

I got a X and Y but I needed to convert to the spacial reference for GPS.

In [4]:
!pip install pyproj
from pyproj import Proj, transform

     |████████████████████████████████| 10.9MB 6.4MB/s eta 0:00:01


In [28]:
from pyproj import Transformer

def function(row):
    transformer = Transformer.from_crs('epsg:3857', 'epsg:4326')
    z2 = transformer.transform(row.X, row.Y)
    return z2

df['GPS'] = df.apply(function, axis=1)

In [29]:
df1 = df
df1[["GPS"]] = df1[["GPS"]].astype("str")
df1['GPS'] = df1['GPS'].str.replace("(", "")
df1['GPS'] = df1['GPS'].str.replace(")", "")
#df1.head()
#df1.dtypes
new = df1['GPS'].str.split(",", n = 1, expand = True) 
df1.head()
# making separate first name column from new data frame 
df1["Latitude"]= new[0] 
df1.head()
#df1.head()
# making separate last name column from new data frame 
df1["Longitude"]= new[1]
# Dropping old Name columns 
df1.drop(columns =['GPS'], inplace = True) 
df1 = df1.drop(['X', 'Y'], axis=1)
df1.head()

,Freguesia,Latitude,Longitude
0,Santo António,38.72134487346995,-9.148808060562017
1,Parque das Nações,38.774809235541774,-9.095477992530249
2,Marvila,38.74793529763537,-9.110954035254304
3,Ajuda,38.712179538844204,-9.19865159430916
4,Areeiro,38.741380367314136,-9.13352482861989


At this point I had the GPS coordenates for each parish of Lisbon. I could start to cross information with Foursquare.

## Analyzing the info

I will get the informatio from Foursquare and cross with the information gathered about the Lisbon parishes. 

In [7]:
CLIENT_ID = 'KSAQZZJC23YINY1EVGBP2O1FI1IWWNEAV4WTZSWW2LIWEW5M' # your Foursquare ID
CLIENT_SECRET = 'VEIFWQ4COHJI31OWZSDTTG0FWCA3NS34UI45P2PXZ5Y4VJ5K' # your Foursquare Secret
VERSION = '20180604'

In [30]:
# the function to retrieve data from Foursquare
def getNearbyVenues(names, latitudes, longitudes):
    radius=2000
    LIMIT=100
# filter by restaurant    
    query='restaurant'
#   category_id = '4d4b7105d754a06374d81259'
# filter by restaurant    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            query)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['location']['formattedAddress']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Address']
    
    return(nearby_venues)

In [31]:
# parish treated 
lisbon_venues = getNearbyVenues(names=df1['Freguesia'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )

Santo António
Parque das Nações
Marvila
Ajuda
Areeiro
Santa Maria Maior
Alvalade
Belém
Estrela
Arroios
Santa Clara
Avenidas Novas
Carnide
São Domingos de Benfica
Beato
Campolide
Alcântara
Campo de Ourique
São Vicente
Olivais
Misericórdia
Lumiar
Penha de França
Benfica


In [33]:
# filter for a parish
print(lisbon_venues.shape)
lisbon_venues.head()
condition = lisbon_venues['Neighborhood'] == 'Santo António'
filtro = lisbon_venues[condition]
print(filtro.shape)
filtro.head(5)

(2199, 8)
(100, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Address
0,Santo António,38.72134487346995,-9.148808060562017,Seen,38.721043,-9.146938,Restaurant,"[Hotel Tivoli, 1250 Lisboa, Portugal]"
1,Santo António,38.72134487346995,-9.148808060562017,Rubro,38.722319,-9.146024,Spanish Restaurant,"[R. Rodrigues Sampaio, 35, 1150-278 Lisboa, Po..."
2,Santo António,38.72134487346995,-9.148808060562017,JNcQUOI,38.720241,-9.145395,Portuguese Restaurant,"[Avenida da Liberdade, 182-184, 1250-146 Lisbo..."
3,Santo António,38.72134487346995,-9.148808060562017,Chimera,38.719867,-9.149094,Modern European Restaurant,"[Rua do Salitre, 131B, Lisboa, Portugal]"
4,Santo António,38.72134487346995,-9.148808060562017,Local - Your Healthy Kitchen,38.724071,-9.147359,Poke Place,"[1150 Lisboa, Portugal]"


In [34]:
#get the zip-code and remove the address
lisbon_venues['ZipCode'] = lisbon_venues['Address'].astype(str).str.extract('(\d{4}\-?\d{0,3})')
lisbon_venues = lisbon_venues.drop(['Address'], axis=1)
lisbon_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,ZipCode
0,Santo António,38.72134487346995,-9.148808060562017,Seen,38.721043,-9.146938,Restaurant,1250
1,Santo António,38.72134487346995,-9.148808060562017,Rubro,38.722319,-9.146024,Spanish Restaurant,1150-278
2,Santo António,38.72134487346995,-9.148808060562017,JNcQUOI,38.720241,-9.145395,Portuguese Restaurant,1250-146
3,Santo António,38.72134487346995,-9.148808060562017,Chimera,38.719867,-9.149094,Modern European Restaurant,NaN
4,Santo António,38.72134487346995,-9.148808060562017,Local - Your Healthy Kitchen,38.724071,-9.147359,Poke Place,1150


At this point I will work with seafood restaurants. I will collect the distance for the neariest seafood restaurants.

In [36]:
# filter for a seafood
lisbon_venues_seafood = lisbon_venues[lisbon_venues['Venue Category'].str.contains('Seafood')]
print(lisbon_venues_seafood.shape)
lisbon_venues_seafood.head(5)

(66, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,ZipCode
46,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Sem Vergonha,38.717711,-9.156884,Seafood Restaurant,1250
54,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Ramiro,38.720347,-9.135703,Seafood Restaurant,1150-007
67,Santo António,38.72134487346995,-9.148808060562017,Sea Me - Peixaria Moderna,38.710706,-9.144251,Seafood Restaurant,1200-241
106,Parque das Nações,38.774809235541774,-9.095477992530249,D' Bacalhau,38.771297,-9.092237,Seafood Restaurant,1900-356
113,Parque das Nações,38.774809235541774,-9.095477992530249,Senhor Peixe,38.771084,-9.092245,Seafood Restaurant,1990-096


In [13]:
from math import sin, cos, sqrt, atan2, radians

In [37]:
lisbon_venues_seafood.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,ZipCode
46,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Sem Vergonha,38.717711,-9.156884,Seafood Restaurant,1250
54,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Ramiro,38.720347,-9.135703,Seafood Restaurant,1150-007
67,Santo António,38.72134487346995,-9.148808060562017,Sea Me - Peixaria Moderna,38.710706,-9.144251,Seafood Restaurant,1200-241
106,Parque das Nações,38.774809235541774,-9.095477992530249,D' Bacalhau,38.771297,-9.092237,Seafood Restaurant,1900-356
113,Parque das Nações,38.774809235541774,-9.095477992530249,Senhor Peixe,38.771084,-9.092245,Seafood Restaurant,1990-096


Collect the distance for the neariest seafood restaurants.

In [38]:
# get the distance for the nearest seafood restaurant
distances_to_seafood_restaurant = []
R = 6373000.0

for (idx, row) in lisbon_venues_seafood.iterrows():
    min_distance = 10000
    for (idx, line) in lisbon_venues_seafood.iterrows():
        
        lat1 = radians(row.loc['Venue Latitude'])
        lon1 = radians(row.loc['Venue Longitude'])
        lat2 = radians(line.loc['Venue Latitude'])
        lon2 = radians(line.loc['Venue Longitude'])
        
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        
        a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        
        distance = R * c
        
        if distance < min_distance and distance > 0:
            min_distance = distance
#           print(distance)
            
    distances_to_seafood_restaurant.append(min_distance)

lisbon_venues_seafood['Distance to Seafood'] = distances_to_seafood_restaurant
lisbon_venues_seafood.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,ZipCode,Distance to Seafood
46,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Sem Vergonha,38.717711,-9.156884,Seafood Restaurant,1250,573.910765
54,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Ramiro,38.720347,-9.135703,Seafood Restaurant,1150-007,857.705045
67,Santo António,38.72134487346995,-9.148808060562017,Sea Me - Peixaria Moderna,38.710706,-9.144251,Seafood Restaurant,1200-241,795.252564
106,Parque das Nações,38.774809235541774,-9.095477992530249,D' Bacalhau,38.771297,-9.092237,Seafood Restaurant,1900-356,23.639360
113,Parque das Nações,38.774809235541774,-9.095477992530249,Senhor Peixe,38.771084,-9.092245,Seafood Restaurant,1990-096,23.639360


In [39]:
print('Average distance to closest Seafood restaurant:', lisbon_venues_seafood['Distance to Seafood'].mean())

Average distance to closest Seafood restaurant: 665.0128429187081


In [17]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
from folium import plugins
from folium.plugins import HeatMap

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [40]:
#preparing the information for the map; extracting only lat and lon data
seafood_latlons = lisbon_venues_seafood.drop(columns =['Neighborhood','Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Category', 'ZipCode', 'Distance to Seafood'], inplace = False) 
seafood_latlons.head()

,Venue,Venue Latitude,Venue Longitude
46,Cervejaria Sem Vergonha,38.717711,-9.156884
54,Cervejaria Ramiro,38.720347,-9.135703
67,Sea Me - Peixaria Moderna,38.710706,-9.144251
106,D' Bacalhau,38.771297,-9.092237
113,Senhor Peixe,38.771084,-9.092245


In [41]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors

In [42]:
# geo info about Lisbon
address = 'Lisbon, PT'
geolocator = Nominatim(user_agent="lisbon")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lisbon are {}, {}.'.format(latitude, longitude))

lisbon_center = [latitude, longitude]

print(lisbon_center)

The geograpical coordinate of Lisbon are 38.7077507, -9.1365919.
[38.7077507, -9.1365919]


This is the map of seafood restaurants in Lisbon.

In [44]:
# map with seafood restaurantes in Lisbon
map_lisbon = folium.Map(location=lisbon_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_lisbon) #cartodbpositron cartodbdark_matter
folium.Marker(lisbon_center).add_to(map_lisbon)
folium.Circle(lisbon_center, radius=3000, fill=False, color='green').add_to(map_lisbon)
folium.Circle(lisbon_center, radius=6000, fill=False, color='green').add_to(map_lisbon)
folium.Circle(lisbon_center, radius=9000, fill=False, color='green').add_to(map_lisbon)

#for lat1, lon1, name1 in zip(lisbon_venues['Venue Latitude'], lisbon_venues['Venue Longitude'], lisbon_venues['Venue']):
#    label1 = folium.Popup(name1, parse_html=True)
#    print(label1)
#    folium.CircleMarker(
#        [lat1, lon1],
#        radius=5,
#        popup=label1,
#        color='red',
#        fill=True,
#        fill_color='red',
#        fill_opacity=0.7).add_to(map_lisbon)

for lat, lon, name in zip(seafood_latlons['Venue Latitude'], seafood_latlons['Venue Longitude'], seafood_latlons['Venue']):
    label = folium.Popup(name, parse_html=True)
    print(label)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_lisbon)

map_lisbon

To see of many restaurants are in the area I will count the number of restaurants near by (<250m) each seafood restaurants. 

In [45]:
# counting the number of restaurants in less than 250m
count_restaurant = []
R = 6373000.0
count = 0

for (idx, row) in lisbon_venues_seafood.iterrows():
    min_distance = 250
    count = 0
    for (idx, line) in lisbon_venues.iterrows():
        
        lat1 = radians(row.loc['Venue Latitude'])
        lon1 = radians(row.loc['Venue Longitude'])
        lat2 = radians(line.loc['Venue Latitude'])
        lon2 = radians(line.loc['Venue Longitude'])
        
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        
        a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        
        distance = R * c
        
        if distance < min_distance and distance > 0:
            count += 1
#           print(distance)
            
#    return count

    count_restaurant.append(count)

lisbon_venues_seafood['Restaurants NearBy'] = count_restaurant
lisbon_venues_seafood.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,ZipCode,Distance to Seafood,Restaurants NearBy
46,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Sem Vergonha,38.717711,-9.156884,Seafood Restaurant,1250,573.910765,3
54,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Ramiro,38.720347,-9.135703,Seafood Restaurant,1150-007,857.705045,12
67,Santo António,38.72134487346995,-9.148808060562017,Sea Me - Peixaria Moderna,38.710706,-9.144251,Seafood Restaurant,1200-241,795.252564,48
106,Parque das Nações,38.774809235541774,-9.095477992530249,D' Bacalhau,38.771297,-9.092237,Seafood Restaurant,1900-356,23.639360,6
113,Parque das Nações,38.774809235541774,-9.095477992530249,Senhor Peixe,38.771084,-9.092245,Seafood Restaurant,1990-096,23.639360,7


The target is tourists, so it's important to be near other restaurantes, but not to many. So I will focus on the seafood restaurants with 5 to 25 restaurants in the area and with noother seafood restautarant less than 500m.

In [51]:
good_res_count = np.array((lisbon_venues_seafood['Restaurants NearBy']>=5) & (lisbon_venues_seafood['Restaurants NearBy']<=25))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_seaf_distance = np.array(lisbon_venues_seafood['Distance to Seafood']>=500)
print('Locations with no Seafood restaurants within 500m:', good_seaf_distance.sum())

good_locations = np.logical_and(good_res_count, good_seaf_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = lisbon_venues_seafood[good_locations]
df_good_locations

Locations with no more than two restaurants nearby: 38
Locations with no Seafood restaurants within 500m: 41
Locations with both conditions met: 15


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,ZipCode,Distance to Seafood,Restaurants NearBy
54,Santo António,38.72134487346995,-9.148808060562017,Cervejaria Ramiro,38.720347,-9.135703,Seafood Restaurant,1150-007,857.705045,12
564,Santa Maria Maior,38.708311728166244,-9.132403842442532,Cervejaria Ramiro,38.720347,-9.135703,Seafood Restaurant,1150-007,857.705045,12
792,Estrela,38.70376841297158,-9.163457993150573,O Palácio,38.706357,-9.173442,Seafood Restaurant,1350,861.396141,15
801,Estrela,38.70376841297158,-9.163457993150573,5 Oceanos,38.699654,-9.178412,Seafood Restaurant,NaN,861.396141,14
881,Arroios,38.72765789247821,-9.137940442549162,Cervejaria Ramiro,38.720347,-9.135703,Seafood Restaurant,1150-007,857.705045,12
963,Arroios,38.72765789247821,-9.137940442549162,Cervejaria Ribadouro,38.719728,-9.145554,Seafood Restaurant,1245-896,857.705045,22
1212,São Domingos de Benfica,38.746451578395,-9.17637255777658,Restaurante Marquês de Palma,38.747058,-9.170257,Seafood Restaurant,1600-130,2376.987677,6
1436,Campolide,38.731158538319484,-9.167269596968547,Restaurante Marquês de Palma,38.747058,-9.170257,Seafood Restaurant,1600-130,2376.987677,6
1462,Alcântara,38.709600048975645,-9.183388632388784,5 Oceanos,38.699654,-9.178412,Seafood Restaurant,NaN,861.396141,14
1465,Alcântara,38.709600048975645,-9.183388632388784,O Palácio,38.706357,-9.173442,Seafood Restaurant,1350,861.396141,15


In [52]:
map_lisbon = folium.Map(location=lisbon_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_lisbon) #cartodbpositron cartodbdark_matter
folium.Marker(lisbon_center).add_to(map_lisbon)
#folium.Circle(lisbon_center, radius=3000, fill=False, color='green').add_to(map_lisbon)
#folium.Circle(lisbon_center, radius=6000, fill=False, color='green').add_to(map_lisbon)
#folium.Circle(lisbon_center, radius=9000, fill=False, color='green').add_to(map_lisbon)

#for lat1, lon1, name1 in zip(lisbon_venues['Venue Latitude'], lisbon_venues['Venue Longitude'], lisbon_venues['Venue']):
#    label1 = folium.Popup(name1, parse_html=True)
#    print(label1)
#    folium.CircleMarker(
#        [lat1, lon1],
#        radius=5,
#        popup=label1,
#        color='red',
#        fill=True,
#        fill_color='red',
#        fill_opacity=0.7).add_to(map_lisbon)

for lat, lon, name in zip(df_good_locations['Venue Latitude'], df_good_locations['Venue Longitude'], df_good_locations['Venue']):
    label = folium.Popup(name, parse_html=True)
    print(label)
    folium.Circle([lat, lon], radius=500, fill=False, color='green').add_to(map_lisbon)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_lisbon)

map_lisbon

In [77]:
candidate_area_addresses = df_good_locations ['ZipCode']
res = [] 
for i in candidate_area_addresses: 
    if i not in res: 
        res.append(i) 

res = [i for i in res if i is not np.nan]

candidate_area_addresses =  res
candidate_area_addresses = pd.DataFrame(candidate_area_addresses)
candidate_area_addresses.rename(columns={0: 'ZipCode'}, inplace=True) 

print('==============================================================')
print('ZipCode of centers of areas recommended for further analysis')
print('==============================================================\n')
candidate_area_addresses


ZipCode of centers of areas recommended for further analysis



,ZipCode
0,1150-007
1,1350
2,1245-896
3,1600-130
4,1000-053
